This module attempts to solve Sudokus

In [70]:
# http://www.websudoku.com/?level=1&set_id=3079647469
# row-represented
input_list = [[0,0,0,0,6,0,0,7,8],
              [0,4,7,0,0,3,5,0,6],
              [6,9,0,0,0,7,0,0,0],
              [9,0,0,6,0,1,3,4,0],
              [5,7,0,0,0,0,0,8,1],
              [0,3,1,5,0,9,0,0,7],
              [0,0,0,1,0,0,0,3,2],
              [7,0,4,2,0,0,6,1,0],
              [1,2,0,0,9,0,0,0,0]]

In [38]:
# HARD: http://www.websudoku.com/?level=3&set_id=3365344417
input_list = [[5,0,0,0,2,0,3,0,0], 
              [0,0,0,7,0,0,0,0,8],
              [0,1,0,0,9,5,7,0,6],
              [9,3,0,0,0,0,2,0,0],
              [0,0,0,0,5,0,0,0,0],
              [0,0,6,0,0,0,0,1,3],
              [7,0,2,4,1,0,0,8,0],
              [1,0,0,0,0,8,0,0,0],
              [0,0,3,0,6,0,0,0,7]]       

In [48]:
# HARD: http://www.websudoku.com/?level=3&set_id=2739385673
input_list = [[8,0,5,0,0,1,0,0,7], 
              [0,2,7,0,0,0,4,0,0],
              [0,0,0,7,0,8,2,0,0],
              [0,6,0,3,0,0,0,2,0],
              [0,0,1,0,0,0,7,0,0],
              [0,7,0,0,0,9,0,5,0],
              [0,0,8,6,0,3,0,0,0],
              [0,0,3,0,0,0,6,1,0],
              [7,0,0,9,0,0,3,0,8]]       

In [80]:
# EVIL: http://www.websudoku.com/?level=4&set_id=7767782257
input_list = [[0,3,0,0,0,4,9,0,0], 
              [2,0,0,0,0,0,8,5,0],
              [0,0,0,9,0,0,0,4,0],
              [0,9,0,4,5,6,1,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,5,1,8,7,0,6,0],
              [0,7,0,0,0,1,0,0,0],
              [0,6,4,0,0,0,0,0,5],
              [0,0,2,6,0,0,0,7,0]]       

In [69]:
def row(x, row_idx, col_idx, exclude_self=False):
    if exclude_self:
        result = [entry for i, entry in enumerate(x[row_idx]) if i != col_idx]
    else:
        result = x[row_idx]
    return result

def col(x, row_idx, col_idx, exclude_self=False):
    if exclude_self:
        result = [entry[col_idx] for i, entry in enumerate(x) if i != row_idx]
    else:
        result = [entry[col_idx] for entry in x]
    return result

def block(x, row_idx, col_idx, exclude_self=False):
    start_row = int(row_idx / 3) * 3
    end_row = start_row + 3
    
    start_col = int(col_idx / 3) * 3
    end_col = start_col + 3
    
    if exclude_self:
        result = [x[i][j] for i in range(start_row, end_row) for j in range(start_col, end_col) if (i != row_idx or j != col_idx)]
    else:        
        result = [x[i][j] for i in range(start_row, end_row) for j in range(start_col, end_col)]
    return result

In [3]:
def possible_values(input_list, row_idx, col_idx):
    row_values = row(input_list, row_idx, col_idx)
    col_values = col(input_list, row_idx, col_idx)
    block_values = block(input_list, row_idx, col_idx)
    
    cell_value = input_list[row_idx][col_idx]
    if cell_value > 0:
        return [cell_value]
    return [i for i in range(1, 10) if not ((i in row_values) or (i in col_values) or (i in block_values))]

In [4]:
def make_candidates(input_list):
    candidates = []
    for i in range(len(input_list)):
        row_res = []
        for j in range(len(input_list[0])):
            row_res.append(possible_values(input_list, i, j))
        candidates.append(row_res)
    return candidates

In [85]:
def fill_values(input_list, candidates_all):
    for i in range(len(input_list)):
        for j in range(len(input_list[0])):
            value_set = False
            if input_list[i][j] == 0 and len(candidates_all[i][j]) == 1:
                input_list[i][j] = candidates_all[i][j][0]
                value_set = True
                continue
            
            candidate_blocks = block(candidates_all, i, j, exclude_self=True)
            candidate_rows = row(candidates_all, i, j, exclude_self=True)
            candidate_cols = col(candidates_all, i, j, exclude_self=True)
            for val in candidates_all[i][j]:
                val_in_block = [val in entry for entry in candidate_blocks]
                val_in_row = [val in entry for entry in candidate_rows]
                val_in_col = [val in entry for entry in candidate_cols]
                if True not in val_in_block or True not in val_in_row or True not in val_in_col:
                    input_list[i][j] = val
                    value_set = True
                    break
                                    

In [6]:
def has_empty_cell(input_list):
    for row in input_list:
        if 0 in row:
            return True
    return False

In [83]:
%%time
num_iter = 0
while has_empty_cell(input_list) and num_iter < 2000:
    fill_values(input_list, make_candidates(input_list))
    num_iter += 1

CPU times: user 3.1 s, sys: 52 ms, total: 3.16 s
Wall time: 3.1 s


In [79]:
make_candidates(input_list)

[[[1, 5, 6, 7, 8],
  [3],
  [1, 6, 7, 8],
  [2, 5, 8],
  [1, 2, 6],
  [4],
  [9],
  [1, 2],
  [1, 2, 6]],
 [[2], [4], [9], [7], [1, 6], [3], [8], [5], [1, 6]],
 [[1, 5, 6, 8],
  [1, 5, 8],
  [1, 6, 8],
  [9],
  [1, 2, 6],
  [2, 5, 8],
  [7],
  [4],
  [3]],
 [[3, 7, 8], [9], [3, 7, 8], [4], [5], [6], [1], [2, 3, 8], [2, 7, 8]],
 [[1, 3, 4, 6, 7, 8],
  [1, 8],
  [1, 3, 6, 7, 8],
  [2, 3],
  [2, 3, 9],
  [2, 9],
  [5],
  [2, 3, 8],
  [2, 4, 7, 8]],
 [[3, 4], [2], [5], [1], [8], [7], [3, 4], [6], [9]],
 [[3, 5, 8, 9],
  [7],
  [3, 8],
  [2, 3, 5, 8],
  [2, 3, 4, 9],
  [1],
  [6],
  [3, 8, 9],
  [4, 8]],
 [[1, 3, 8, 9], [6], [4], [3, 8], [7], [8, 9], [2], [1, 3, 8, 9], [5]],
 [[1, 3, 5, 8, 9],
  [1, 5, 8],
  [2],
  [6],
  [3, 4, 9],
  [5, 8, 9],
  [3, 4],
  [7],
  [1, 4, 8]]]

In [72]:
input_list

[[0, 3, 0, 0, 0, 4, 9, 0, 0],
 [2, 4, 9, 7, 0, 3, 8, 5, 0],
 [0, 0, 0, 9, 0, 0, 7, 4, 0],
 [0, 9, 0, 4, 5, 6, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 5, 0, 0],
 [0, 2, 5, 1, 8, 7, 0, 6, 9],
 [0, 7, 0, 0, 0, 1, 0, 0, 0],
 [0, 6, 4, 0, 7, 0, 0, 0, 5],
 [0, 0, 2, 6, 0, 0, 0, 7, 0]]

In [71]:
fill_values(input_list, make_candidates(input_list))

> <ipython-input-70-4f43af1fb654>(14)fill_values()
-> candidate_blocks = block(candidates_all, i, j, exclude_self=True)
(Pdb) l
  9  	                value_set = True
 10  	                continue
 11  	            
 12  	            if i == 5 and j == 8:
 13  	                pdb.set_trace()
 14  ->	            candidate_blocks = block(candidates_all, i, j, exclude_self=True)
 15  	            candidate_rows = row(candidates_all, i, j, exclude_self=True)
 16  	            candidate_cols = col(candidates_all, i, j, exclude_self=True)
 17  	            for val in candidates_all[i][j]:
 18  	                val_in_block = [val in entry for entry in candidate_blocks]
 19  	                val_in_row = [val in entry for entry in candidate_rows]
(Pdb) n
> <ipython-input-70-4f43af1fb654>(15)fill_values()
-> candidate_rows = row(candidates_all, i, j, exclude_self=True)
(Pdb) n
> <ipython-input-70-4f43af1fb654>(16)fill_values()
-> candidate_cols = col(candidates_all, i, j, exclude_self=True)
